In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Mar 19 22:35:34 2025

@author: ACER
"""
!pip install ortools
import numpy as np
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

# Titik-titik antar-jemput (koordinat atau jarak antar titik)
titik_antar_jemput = [
    (0, 0), (1, 2), (3, 1), (2, 4), (4, 3)  # Contoh koordinat 5 titik
]

# Jumlah pekerja di setiap titik antar-jemput
jumlah_pekerja = [0, 10, 15, 8, 12, 7]  # Tambahkan 0 untuk depot

# Jumlah armada bus dan kapasitasnya
jumlah_armada = 3
kapasitas_bus = [20, 25, 30]

# Time-windows untuk antar-jemput (dalam satuan waktu)
time_windows = [
    (0, 10), (5, 15), (10, 20), (15, 25), (20, 30)
]

# Menyesuaikan jumlah node agar sesuai dengan depot
manager = pywrapcp.RoutingIndexManager(len(titik_antar_jemput), jumlah_armada, 0)
routing = pywrapcp.RoutingModel(manager)

# Mendefinisikan fungsi jarak antar titik sebagai estimasi waktu perjalanan
def distance_callback(from_index, to_index):
    """Menghitung jarak Euclidean sebagai estimasi waktu perjalanan."""
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    distance = np.linalg.norm(np.array(titik_antar_jemput[from_node]) - np.array(titik_antar_jemput[to_node]))
    return int(distance)  # OR-Tools mengharapkan nilai integer

transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Menambahkan kendala kapasitas bus
def demand_callback(from_index):
    from_node = manager.IndexToNode(from_index)
    return jumlah_pekerja[from_node]

demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
routing.AddDimensionWithVehicleCapacity(
    demand_callback_index, 0, kapasitas_bus, True, 'Capacity')

# Menambahkan kendala time windows
routing.AddDimension(
    transit_callback_index,  # Callback jarak yang juga mewakili waktu
    30,  # Maksimum waktu tunggu
    100,  # Waktu maksimal perjalanan untuk setiap kendaraan
    False,  # Jangan paksa waktu mulai menjadi nol
    "Time")

time_dimension = routing.GetDimensionOrDie("Time")

# Set time window untuk setiap lokasi
for location_idx, time_window in enumerate(time_windows):
    index = manager.NodeToIndex(location_idx)
    time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])

# Set time window untuk depot
for vehicle_id in range(jumlah_armada):
    index = routing.Start(vehicle_id)
    time_dimension.CumulVar(index).SetRange(time_windows[0][0], time_windows[0][1])

# Menentukan fungsi tujuan
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

# Menjalankan solver
solution = routing.SolveWithParameters(search_parameters)

# Menampilkan solusi jika ditemukan
if solution:
    for vehicle_id in range(jumlah_armada):
        index = routing.Start(vehicle_id)
        route = []
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route.append(node_index)
            index = solution.Value(routing.NextVar(index))
        route.append(manager.IndexToNode(index))  # Tambahkan titik akhir
        print(f"bus route {vehicle_id + 1}: {route}")
else:
    print("Tidak ditemukan solusi yang valid.")


bus route 1: [0, 0]
bus route 2: [0, 2, 0]
bus route 3: [0, 1, 3, 4, 0]
